# Get Wienerschnitzel locations

#### Load Python tools and Jupyter config

In [1]:
%load_ext lab_black

In [2]:
import json
import requests
import warnings
import pandas as pd
import altair as alt
from time import sleep
import geopandas as gpd
from random import randint
from tqdm.notebook import tqdm, trange

In [3]:
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 1000
pd.options.display.max_colwidth = None
warnings.simplefilter(action="ignore")

In [4]:
place = "wienerschnitzel"

## Get data

#### Headers for request

In [5]:
cookies = {
    "__cf_bm": "2wsr23arSZqxmZisPaEHe1q3Xm9nv4gt6jaGrRAKC5Q-1708183357-1.0-ARpjqN3wVC6kmTqmgNhS9SNHSRXgMxbTEcVx6Wp8Mey8iYELhL7hX64IduCqMk4GbDiAA0v6HrJA6/JkYSCkQ5k=",
    "_gid": "GA1.2.708023445.1708183359",
    "_gat_UA-2492191-1": "1",
    "PHPSESSID": "992cb2f2f5b47b0f5d74a5c281cf68e3",
    "_fbp": "fb.1.1708183358709.281967528",
    "cookie_notice_accepted": "true",
    "_gcl_au": "1.1.755963563.1708183365",
    "_ga_D8ZEN8D9QP": "GS1.1.1708183358.1.1.1708183382.0.0.0",
    "_ga": "GA1.2.294461605.1708183359",
}

headers = {
    "authority": "www.wienerschnitzel.com",
    "accept": "*/*",
    "accept-language": "en-US,en;q=0.9,es;q=0.8",
    "content-type": "application/x-www-form-urlencoded; charset=UTF-8",
    # 'cookie': '__cf_bm=2wsr23arSZqxmZisPaEHe1q3Xm9nv4gt6jaGrRAKC5Q-1708183357-1.0-ARpjqN3wVC6kmTqmgNhS9SNHSRXgMxbTEcVx6Wp8Mey8iYELhL7hX64IduCqMk4GbDiAA0v6HrJA6/JkYSCkQ5k=; _gid=GA1.2.708023445.1708183359; _gat_UA-2492191-1=1; PHPSESSID=992cb2f2f5b47b0f5d74a5c281cf68e3; _fbp=fb.1.1708183358709.281967528; cookie_notice_accepted=true; _gcl_au=1.1.755963563.1708183365; _ga_D8ZEN8D9QP=GS1.1.1708183358.1.1.1708183382.0.0.0; _ga=GA1.2.294461605.1708183359',
    "origin": "https://www.wienerschnitzel.com",
    "referer": "https://www.wienerschnitzel.com/location-results/?address=90066",
    "sec-ch-ua": '"Not A(Brand";v="99", "Google Chrome";v="121", "Chromium";v="121"',
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": '"macOS"',
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-origin",
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36",
    "x-requested-with": "XMLHttpRequest",
}

#### Parameters for the request, including a wide radius and high record return limit. Too easy. 

In [6]:
params = "address=90066&formdata=nameSearch%3D%26addressInput%3D90066%26addressInputCity%3D%26addressInputState%3D%26addressInputCountry%3D%26ignore_radius%3D0&lat=34.0050173&lng=-118.4338345&name=&options%5Bbubblelayout%5D=%3Cdiv+id%3D%22sl_info_bubble%22+class%3D%22%5Bslp_location+featured%5D%22%3E%0A%3Cspan+id%3D%22slp_bubble_name%22%3E%3Cstrong%3E%5Bslp_location+name++suffix++br%5D%3C%2Fstrong%3E%3C%2Fspan%3E%0A%3Cspan+id%3D%22slp_bubble_address%22%3E%5Bslp_location+address+++++++suffix++br%5D%3C%2Fspan%3E%0A%3Cspan+id%3D%22slp_bubble_address2%22%3E%5Bslp_location+address2++++++suffix++br%5D%3C%2Fspan%3E%0A%3Cspan+id%3D%22slp_bubble_city%22%3E%5Bslp_location+city++++++++++suffix++comma%5D%3C%2Fspan%3E%0A%3Cspan+id%3D%22slp_bubble_state%22%3E%5Bslp_location+state+suffix++++space%5D%3C%2Fspan%3E%0A%3Cspan+id%3D%22slp_bubble_zip%22%3E%5Bslp_location+zip+suffix++br%5D%3C%2Fspan%3E%0A%3Cspan+id%3D%22slp_bubble_country%22%3E%3Cspan+id%3D%22slp_bubble_country%22%3E%5Bslp_location+country+++++++suffix++br%5D%3C%2Fspan%3E%3C%2Fspan%3E%0A%3Cspan+id%3D%22slp_bubble_directions%22%3E%5Bhtml+br+ifset+directions%5D%0A%5Bslp_option+label_directions+wrap+directions%5D%3C%2Fspan%3E%0A%3Cspan+id%3D%22slp_bubble_website%22%3E%5Bhtml+br+ifset+url%5D%0A%5Bslp_location+url+++++++++++wrap++++website%5D%5Bslp_option+label_website+ifset+url%5D%5Bhtml+closing_anchor+ifset+url%5D%5Bhtml+br+ifset+url%5D%3C%2Fspan%3E%0A%3Cspan+id%3D%22slp_bubble_email%22%3E%5Bslp_location+email+++++++++wrap++++mailto+%5D%5Bslp_option+label_email+ifset+email%5D%5Bhtml+closing_anchor+ifset+email%5D%5Bhtml+br+ifset+email%5D%3C%2Fspan%3E%0A%3Cspan+id%3D%22slp_bubble_phone%22%3E%5Bhtml+br+ifset+phone%5D%0A%3Cspan+class%3D%22location_detail_label%22%3E%5Bslp_option+++label_phone+++ifset+++phone%5D%3C%2Fspan%3E%5Bslp_location+phone+++++++++suffix++++br%5D%3C%2Fspan%3E%0A%3Cspan+id%3D%22slp_bubble_fax%22%3E%3Cspan+class%3D%22location_detail_label%22%3E%5Bslp_option+++label_fax+++++ifset+++fax++%5D%3C%2Fspan%3E%5Bslp_location+fax+++++++++++suffix++++br%5D%3Cspan%3E%0A%3Cspan+id%3D%22slp_bubble_description%22%3E%3Cspan+id%3D%22slp_bubble_description%22%3E%5Bhtml+br+ifset+description%5D%0A%5Bslp_location+description+raw%5D%3C%2Fspan%3E%5Bhtml+br+ifset+description%5D%3C%2Fspan%3E%0A%3Cspan+id%3D%22slp_bubble_hours%22%3E%5Bhtml+br+ifset+hours%5D%0A%3Cspan+class%3D%22location_detail_label%22%3E%5Bslp_option+++label_hours+++ifset+++hours%5D%3C%2Fspan%3E%0A%3Cspan+class%3D%22location_detail_hours%22%3E%5Bslp_location+hours+++++++++suffix++++br%5D%3C%2Fspan%3E%3C%2Fspan%3E%0A%3Cspan+id%3D%22slp_bubble_img%22%3E%5Bhtml+br+ifset+img%5D%0A%5Bslp_location+image+++++++++wrap++++img%5D%3C%2Fspan%3E%0A%3Cspan+id%3D%22slp_tags%22%3E%5Bslp_location+tags%5D%3C%2Fspan%3E%0A%3C%2Fdiv%3E&options%5Bdistance_unit%5D=miles&options%5Bimmediately_show_locations%5D=0&options%5Binitial_radius%5D=200&options%5Binitial_results_returned%5D=1000&options%5Blabel_directions%5D=Get+Directions&options%5Blabel_fax%5D=Fax%3A+&options%5Blabel_email%5D=Email&options%5Blabel_phone%5D=Phone%3A+&options%5Blabel_website%5D=More+Info&options%5Bmap_center%5D=United+States&options%5Bmap_center_lat%5D=37.09024&options%5Bmap_center_lng%5D=-95.712891&options%5Bmap_domain%5D=maps.google.com&options%5Bmap_end_icon%5D=https%3A%2F%2Fwww.wienerschnitzel.com%2Fwp-content%2Fuploads%2Fslp%2Fsaved-icons%2Fws_pin.png&options%5Bmap_home_icon%5D=https%3A%2F%2Fwww.wienerschnitzel.com%2Fwp-content%2Fuploads%2Fslp%2Fsaved-icons%2Fws_home.png&options%5Bmap_type%5D=roadmap&options%5Bmessage_no_results%5D=No+locations+found.&options%5Bno_autozoom%5D=0&options%5Bno_homeicon_at_start%5D=1&options%5Bradii%5D=10%2C25%2C50%2C100%2C200%2C500%2C(5000)&options%5Bslplus_version%5D=4.2.07&options%5Bzoom_level%5D=8&options%5Bzoom_tweak%5D=1&options%5Bignore_radius%5D=0&options%5Btheme%5D=&options%5Bid%5D=&options%5Bhide_search_form%5D=0&options%5Bendicon%5D=&options%5Bhomeicon%5D=&options%5Bonly_with_tag%5D=&options%5Btags_for_pulldown%5D=&options%5Btags_for_dropdown%5D=&options%5Bhide_results%5D=0&options%5Border_by%5D=sl_distance+ASC&options%5Ballow_addy_in_url%5D=0&options%5Bhide_address_entry%5D=0&options%5Bappend_to_search%5D=&options%5Bcity%5D=&options%5Bcountry%5D=&options%5Bstate%5D=&options%5Bcity_selector%5D=hidden&options%5Bcountry_selector%5D=hidden&options%5Bstate_selector%5D=dropdown_addressinput&options%5Bradius_behavior%5D=always_use&options%5Buse_sensor%5D=false&options%5Badd_tel_to_phone%5D=0&options%5Bdisable_initial_directory%5D=0&options%5Bextended_data_version%5D=&options%5Bhide_distance%5D=0&options%5Binstalled_version%5D=4.3.02&options%5Borderby%5D=sl_distance+ASC&options%5Bresultslayout%5D=%3Cdiv+id%3D%22slp_results_%5Bslp_location+id%5D%22+class%3D%22results_entry++%5Bslp_location+featured%5D%22+data-tags%3D%22%5Bslp_location+tags%5D%22%3E%0D%0A%09%3Cdiv+class%3D%22vc_row+wpb_row+vc_row-fluid++%22%3E%3Cdiv+class%3D%22clr%22%3E%0D%0A+++++++++++++%3Cdiv++id%3D%22slp_left_cell_%5Bslp_location+id%5D%22%3E%0D%0A%09%09%3Cdiv+class%3D%22vc_col-sm-2+wpb_column+column_container+location-icon%22%3E%3Cdiv+class%3D%22clr%22%3E%0D%0A%0D%0A%09%09%09%3Cdiv+class%3D%22slp_map_icon%22%3E%3Cimg+src%3D%22%2Fwp-content%2Fuploads%2Fslp%2Fsaved-icons%2Fws_pin.png%22+alt%3D%22%5Bslp_location+name%5D%22+%2F%3E%3C%2Fdiv%3E%0D%0A%0D%0A%09%09%09%3Cdiv%3E%0D%0A%09%09%09%09%3Cspan+class%3D%22location_name%22%3E%5Bslp_location+name%5D%3C%2Fspan%3E%0D%0A%09%09%09%09%3Cspan+class%3D%22location_distance%22%3E%3Cstrong%3E%5Bslp_location+distance_1%5D+mi%3C!--%5Bslp_location+distance_unit%5D--%3E%3C%2Fstrong%3E%3C%2Fspan%3E%0D%0A%09%09%09%3C%2Fdiv%3E%0D%0A%0D%0A%09%09%3C%2Fdiv%3E%3C%2Fdiv%3E%09%0D%0A%09%09%09%0D%0A%09%09%3Cdiv+class%3D%22vc_col-sm-10+wpb_column+column_container%22%3E%3Cdiv+class%3D%22clr%22%3E%0D%0A%09%09%09%3Cdiv+class%3D%22results_row_center_column%22+id%3D%22slp_center_cell_%5Bslp_location+id%5D%22+%3E%0D%0A%3Cdiv+class%3D%22notice%22+data-tags%3D%22%5Bslp_location+tags%5D%22%3E%3C%2Fdiv%3E%0D%0A%09%09%09%09%3Cspan+class%3D%22slp_result_address+slp_result_street%22%3E%5Bslp_location+address%5D%3C%2Fspan%3E%0D%0A%09%09%09%09%3Cspan+class%3D%22slp_result_address+slp_result_street2%22%3E%5Bslp_location+address2%5D%3C%2Fspan%3E%0D%0A%09%09%09%09%3Cspan+class%3D%22slp_result_address+slp_result_citystatezip%22%3E%5Bslp_location+city_state_zip%5D%3C%2Fspan%3E%0D%0A%09%09%09%09%3Cspan+class%3D%22slp_result_address+slp_result_country%22%3E%5Bslp_location+country%5D%3C%2Fspan%3E%0D%0A%09%09%09%09%3Cspan+class%3D%22slp_result_address+slp_result_phone%22%3E%26nbsp%3B%3Ca+href%3D%22tel%3A%5Bslp_location+phone%5D%22+class%3D%22call%22+data-track%3D%22%2Fcall%2Fstore%2F%5Bslp_location+phone%5D%22%3E%5Bslp_location+phone%5D%3C%2Fa%3E%3C%2Fspan%3E%0D%0A%09%09%09%09%3Cspan+class%3D%22slp_result_address+slp_result_fax%22%3E%5Bslp_location+fax%5D%3C%2Fspan%3E%0D%0A%09%09%09%3C%2Fdiv%3E%0D%0A%09%09%09%3Cdiv+class%3D%22results_row_right_column%22++id%3D%22slp_right_cell_%5Bslp_location+id%5D%22++%3E%0D%0A++++++++++++++++++++++++++++++++%0D%0A%09%09%09%09%3Cspan+class%3D%22slp_result_contact+slp_result_website%22%3E%5Bslp_location+web_link%5D%3C%2Fspan%3E%0D%0A%09%09%09%09%3Cspan+class%3D%22slp_result_contact+slp_result_email%22%3E%5Bslp_location+email_link%5D%3C%2Fspan%3E%0D%0A%09%09%09%09%3Cspan+class%3D%22slp_result_contact+slp_result_directions%22%3E%3Ca+href%3D%22https%3A%2F%2F%5Bslp_location+map_domain%5D%2Fmaps%3Fsaddr%3D%5Bslp_location+search_address%5D%26daddr%3D%5Bslp_location+location_address%5D%22+target%3D%22_blank%22+class%3D%22storelocatorlink+directions%22+data-track%3D%22%2Fdirections%2Fstore%2F%5Bslp_location+phone%5D%22%3E%5Bslp_location+directions_text%5D%3C%2Fa%3E%3C%2Fspan%3E%0D%0A%3Cspan+class%3D%22slp_result_contact+slp_result_website+order%22%3E%3Ca+class%3D%22order-online%22%3EOrder+Online%3C%2Fa%3E%3C%2Fspan%3E%0D%0A%09%09%09%09%0D%0A%09%09%09%09%5Bslp_location+iconarray+wrap%3D%22fullspan%22%5D%0D%0A%09%09%09%3C%2Fdiv%3E%0D%0A%09%09%3C%2Fdiv%3E%3C%2Fdiv%3E%0D%0A+++++++++++%3C%2Fdiv%3E%0D%0A%09%09%09%0D%0A%09%3C%2Fdiv%3E%3C%2Fdiv%3E%0D%0A%0D%0A%3C%2Fdiv%3E&options%5Bshow_country%5D=0&options%5Bshow_hours%5D=0&options%5Bfeatured_location_display_type%5D=show_within_radius&options%5Bemail_link_format%5D=label_link&options%5Bpopup_email_title%5D=Send+An+Email&options%5Bpopup_email_from_placeholder%5D=Your+email+address.&options%5Bpopup_email_subject_placeholder%5D=Email+subject+line.&options%5Bpopup_email_message_placeholder%5D=What+do+you+want+to+say%3F&options%5Baddress_autocomplete%5D=none&options%5Baddress_autocomplete_min%5D=3&options%5Bsearchnear%5D=currentmap&options%5Bselector_behavior%5D=use_both&options%5Buse_same_window%5D=on&options%5Bforce_load_js%5D=0&options%5Bmap_region%5D=us&radius=20000&tags=&action=csl_ajax_onload"

#### Make the request

In [7]:
response = requests.post(
    "https://www.wienerschnitzel.com/wp-admin/admin-ajax.php",
    headers=headers,
    data=params,
)

#### Read the json response as a dataframe

In [8]:
src = pd.DataFrame(response.json()["response"])

#### Clean dataframe with just the columns we need

In [9]:
df = (
    src[
        [
            "name",
            "address",
            "city",
            "state",
            "zip",
            "lat",
            "lng",
            "url",
            "phone",
        ]
    ]
    .rename(columns={"lat": "latitude", "lng": "longitude"})
    .copy()
)

In [17]:
len(df)

320

---

## Geography

#### Make it a geodataframe

In [10]:
df_geo = df.copy()

In [11]:
gdf = gpd.GeoDataFrame(
    df_geo, geometry=gpd.points_from_xy(df_geo.longitude, df_geo.latitude)
)

In [12]:
locations_gdf = gdf.set_crs("EPSG:4326").copy()

---

## Maps

#### US states background

In [13]:
from vega_datasets import data

background = (
    alt.Chart(alt.topo_feature(data.us_10m.url, feature="states"))
    .mark_geoshape(fill="#e9e9e9", stroke="white")
    .properties(width=800, height=500, title=f"{place.title()} locations")
    .project("albersUsa")
)

#### Location points map

In [14]:
points = (
    alt.Chart(gdf)
    .mark_circle(size=6, color="#ed1b2e")
    .encode(
        longitude="longitude:Q",
        latitude="latitude:Q",
    )
)

point_map = background + points
point_map.configure_view(stroke=None)

alt.LayerChart(...)

#### Location proportional symbols map

In [15]:
symbols = (
    alt.Chart(gdf)
    .transform_aggregate(
        latitude="mean(latitude)",
        longitude="mean(longitude)",
        count="count()",
        groupby=["state"],
    )
    .mark_circle()
    .encode(
        longitude="longitude:Q",
        latitude="latitude:Q",
        size=alt.Size("count:Q", title="Count by state"),
        color=alt.value("#ed1b2e"),
        tooltip=["state:N", "count:Q"],
    )
    .properties(title=f"Number of {place} in US, by average lon/lat of locations")
)

symbol_map = background + symbols
symbol_map.configure_view(stroke=None)

alt.LayerChart(...)

---

## Exports

#### JSON

In [16]:
df.to_json(
    f"data/processed/{place.lower().replace(' ', '_')}_locations.json",
    indent=4,
    orient="records",
)

OSError: Cannot save file into a non-existent directory: 'data/processed'

#### CSV

In [ ]:
df.to_csv(
    f"data/processed/{place.lower().replace(' ', '_')}_locations.csv", index=False
)

#### GeoJSON

In [ ]:
locations_gdf.to_file(
    f"data/processed/{place.lower().replace(' ', '_')}_locations.geojson",
    driver="GeoJSON",
)